In [9]:
# load data and display schema
df1 = spark.read.format("csv").option("header", "true").load("yellow_tripdata_2015-01.csv")
# display first 5 rows 
display(df1.head(5))

# statistics for all numerical columns
columns = ['passenger_count', 'trip_distance', 'fare_amount']
df1.describe(columns).show()

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
# a) Handle missing values
df_cleaned = df1.na.drop(subset=["fare_amount", "trip_distance", "passenger_count"])

# b) Filter out rows with invalid data
df_cleaned = df_cleaned.filter((df_cleaned["fare_amount"] > 0.0) & (df_cleaned["trip_distance"] > 0.0))

from pyspark.sql.functions import to_timestamp, col
# c) Converting pickup_datetime and dropoff_datetime to timestamp
df_cleaned = df_cleaned.withColumn("tpep_pickup_datetime", to_timestamp(df_cleaned["tpep_pickup_datetime"]))
df_cleaned = df_cleaned.withColumn("tpep_dropoff_datetime", to_timestamp(df_cleaned["tpep_dropoff_datetime"]))

# d) Create new columns: trip duration (minutes) and trip speed (miles/hour)
df_cleaned = df_cleaned.withColumn("trip_duration", (col("tpep_dropoff_datetime").cast("long") - col("tpep_pickup_datetime").cast("long")) / 60)
df_cleaned = df_cleaned.withColumn("trip_speed", col("trip_distance") / (col("trip_duration") / 60))

In [ ]:
from pyspark.sql.functions import avg

# Passenger statistics
passenger_stats = df_cleaned.groupBy("passenger_count") \
    .agg(avg("fare_amount").alias("Average_fare"), avg("trip_distance").alias("Average_TripDistance")) \
    .orderBy("passenger_count", ascending=True)
display(passenger_stats)

# Find busiest times of day
from pyspark.sql.functions import hour
new_df = df_cleaned.withColumn("hours", hour(df_cleaned["tpep_pickup_datetime"]))
busiest_time = new_df.groupBy("hours").count().orderBy("count", ascending=False)
display(busiest_time)

# Neighbourhood analysis
from pyspark.sql.functions import when
df_with_location = df_cleaned.withColumn(
    "neighbourhood",
    when(df_cleaned.pickup_longitude < -74.0060, "West").otherwise("East")
)
neighbourhood_avg = df_with_location.groupBy("neighbourhood").agg(avg("fare_amount").alias("average_fare_amount"))
display(neighbourhood_avg)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convert to Pandas DataFrame for plotting
df_cleaned = df_cleaned.toPandas()

# Plot trip distance histogram
plt.figure(figsize=(20, 12))
plt.hist(df_cleaned['trip_distance'], bins=30, color='blue', edgecolor='black')
plt.title('Histogram of Trip Distance')
plt.xlabel('Trip Distance')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

# Average fare per hour visualization
average_fare_graph = new_df.groupBy("hours") \
    .agg(avg("fare_amount").alias("Average_fare")) \
    .orderBy("hours", ascending=True).toPandas()
plt.figure(figsize=(20, 12))
plt.plot(average_fare_graph['hours'], average_fare_graph['Average_fare'], marker='o')
plt.xlabel('Hours')
plt.ylabel('Average Fare Amount ($)')
plt.title('Average Fare per Hour of the Day')
plt.grid(True)
plt.xticks(range(0, 24))
plt.show()

In [ ]:
from pyspark.sql.functions import col, to_timestamp, floor, lpad, concat, lit, hour, dayofweek, weekofyear

# Trip Duration
df1 = df1.withColumn("tpep_pickup_datetime", to_timestamp("tpep_pickup_datetime"))
df1 = df1.withColumn("tpep_dropoff_datetime", to_timestamp("tpep_dropoff_datetime"))
df1 = df1.withColumn("trip_duration_seconds", (col("tpep_dropoff_datetime").cast("long") - col("tpep_pickup_datetime").cast("long")))

# Hour and Day Extraction
df1 = df1.withColumn("pickup_hour", hour("tpep_pickup_datetime")).withColumn("pickup_day", dayofweek("tpep_pickup_datetime"))

# Weekly Trends
df1 = df1.withColumn("pickup_week", weekofyear("tpep_pickup_datetime"))
weekly_trends = df1.groupBy("pickup_week") \
    .agg((avg("trip_duration_seconds") / 60).alias("avg_trip_duration_minutes")) \
    .orderBy("pickup_week").toPandas()

# Hourly Analysis
hourly_analysis = df1.groupBy("pickup_hour").avg("trip_duration_seconds").orderBy("pickup_hour").toPandas()

# Hotspot Identification
df_filtered = df1.filter((col("pickup_longitude") != 0) & (col("pickup_latitude") != 0))
pickup_hotspots = df_filtered.groupBy("pickup_longitude", "pickup_latitude").count().orderBy("count", ascending=False).toPandas()

# Average Fare by Pickup Location
df_filtered = df_filtered.withColumn("pickup_location", concat(col("pickup_longitude"), lit(", "), col("pickup_latitude")))
top_fare_locations = df_filtered.groupBy("pickup_location").agg(avg("fare_amount").alias("average_fare")).orderBy("average_fare", ascending=False).toPandas()

In [ ]:
df1 = df1.withColumn("trip_distance", col("trip_distance").cast("float"))
df1 = df1.withColumn("fare_amount", col("fare_amount").cast("float"))
trip_duration_distance_corr = df1.stat.corr("trip_duration_seconds", "trip_distance")
trip_duration_fare_corr = df1.stat.corr("trip_duration_seconds", "fare_amount")
trip_distance_fare_corr = df1.stat.corr("trip_distance", "fare_amount")

print("Correlation between Trip Duration and Trip Distance:", trip_duration_distance_corr)
print("Correlation between Trip Duration and Fare Amount:", trip_duration_fare_corr)
print("Correlation between Trip Distance and Fare Amount:", trip_distance_fare_corr)